# lDE30 Image Processing Notebook - Run 1

See './Image_Analysis/Examples/1_Core_Image_Processing.ipynb' for a documented example of notebook usage.

#### Imports

In [ ]:
import trenchripper.trenchripper as tr

import warnings

warnings.filterwarnings(action="once")

import matplotlib

matplotlib.rcParams["figure.figsize"] = [20, 10]

In [ ]:
# addition of active memory manager
import dask
dask.config.set({'distributed.scheduler.active-memory-manager.start': True});
dask.config.set({'distributed.scheduler.worker-ttl': "5m"});
dask.config.set({'distributed.scheduler.allowed-failures': 100});

dask_wd = "/home/de64/scratch/de64/dask"

# Part 0: Flat Field Generation

### Dark Image

flatfieldpath = "/home/de64/scratch/de64/sync_folder/2022-12-26_Flat_Fields_Iris_9_40x/DarkImage_40x_Phase.nd2"
outputpath = "/home/de64/scratch/de64/sync_folder/2022-12-26_Flat_Fields_Iris_9_40x/40x_DarkImage.tiff"
tr.generate_flatfield(flatfieldpath,outputpath)

### Cy5-Penta

flatfieldpath = "/home/de64/scratch/de64/sync_folder/2022-12-07_Flat_Fields_Iris_9_20x/Cy5-Penta_20x_Phase.nd2"
outputpath = "/home/de64/scratch/de64/sync_folder/2022-12-07_Flat_Fields_Iris_9_20x/Cy5-Penta_20x_Ph2_flatfield.tiff"
tr.generate_flatfield(flatfieldpath,outputpath)

### Cy7-Penta

flatfieldpath = "/home/de64/scratch/de64/sync_folder/2022-12-07_Flat_Fields_Iris_9_20x/Cy7-Penta_20x_Phase.nd2"
outputpath = "/home/de64/scratch/de64/sync_folder/2022-12-07_Flat_Fields_Iris_9_20x/Cy7-Penta_20x_Ph2_flatfield.tiff"
tr.generate_flatfield(flatfieldpath,outputpath)

### GFP-Penta

flatfieldpath = "/home/de64/scratch/de64/sync_folder/2022-12-07_Flat_Fields_Iris_9_20x/GFP-Penta_20x_Phase.nd2"
outputpath = "/home/de64/scratch/de64/sync_folder/2022-12-07_Flat_Fields_Iris_9_20x/GFP-Penta_20x_Ph2_flatfield.tiff"
tr.generate_flatfield(flatfieldpath,outputpath)

### MCherry

flatfieldpath = "/home/de64/scratch/de64/sync_folder/2022-12-26_Flat_Fields_Iris_9_20x/mCherry_20x_Phase.nd2"
outputpath = "/home/de64/scratch/de64/sync_folder/2022-12-26_Flat_Fields_Iris_9_20x/mCherry_20x_Ph2_flatfield.tiff"
tr.generate_flatfield(flatfieldpath,outputpath)

In [ ]:
# Luis: Modified to the already-extracted 40X Flat Fields

In [ ]:
import tifffile
from matplotlib import pyplot as plt

In [ ]:
# Changed by Luis to include len(fflist) instead of harcoded "5"

dark_outputpath = "/home/lag36/gutiloluis/FlatFields/2022-12-26_Ti5-40xPh2/40x_DarkImage.tiff"
Cy5_outputpath = "/home/lag36/gutiloluis/FlatFields/2022-12-26_Ti5-40xPh2/40x_Cy5-Penta.tiff"
Cy7_outputpath = "/home/lag36/gutiloluis/FlatFields/2022-12-26_Ti5-40xPh2/40x_Cy7-Penta.tiff"
GFP_outputpath = "/home/lag36/gutiloluis/FlatFields/2022-12-26_Ti5-40xPh2/40x_GFP-Penta.tiff"
Mcherry_outputpath = "/home/lag36/gutiloluis/FlatFields/2022-12-26_Ti5-40xPh2/40x_mCherry.tiff"
YFP_outputpath = "/home/lag36/gutiloluis/FlatFields/2022-12-26_Ti5-40xPh2/40x_YFP.tiff"

fflist = [dark_outputpath,Cy5_outputpath,Cy7_outputpath,GFP_outputpath,Mcherry_outputpath, YFP_outputpath]

fig, axs = plt.subplots(figsize=(15, 20), nrows= 1, ncols = len(fflist))

for i in range(len(fflist)):
    axs[i].imshow(tifffile.imread(fflist[i]))
    # axs[i].set_title(fflist[i].split('/')[-1].split('_')[0]) # 
    axs[i].set_title(fflist[i].split('/')[-1].split('_')[1]) # CHANGED BY LUIS
plt.tight_layout()

# Part 1: Growth/Division

#### Specify Paths

In [ ]:
headpath = "/home/de64/scratch/de64/sync_folder/2023-05-06_lDE30_Run_1/Growth_Division"
nd2file = "/home/de64/scratch/de64/sync_folder/2023-05-06_lDE30_Run_1/Experiment.nd2"

## Extract to hdf5 files

#### Start Dask Workers

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="1:00:00",
    local=False,
    n_workers=200,
    n_workers_min=50,
    memory="8GB",
    working_directory=dask_wd
)
dask_controller.startdask()

In [ ]:
dask_controller.displaydashboard()

##### Perform Extraction

In [ ]:
hdf5_extractor = tr.ndextract.hdf5_fov_extractor(
    nd2file,
    headpath,
    tpts_per_file=50,
    ignore_fovmetadata=False,
    nd2reader_override={"z_levels": [], "z_coordinates": []},
)

##### Extraction Parameters

In [ ]:
hdf5_extractor.inter_set_params()

#### List of paths to flat fields
 - "/home/lag36/gutiloluis/FlatFields/2022-12-26_Ti5-40xPh2/40x_mCherry.tiff"
 - "/home/lag36/gutiloluis/FlatFields/2022-12-26_Ti5-40xPh2/40x_YFP.tiff"
 - "/home/lag36/gutiloluis/FlatFields/2022-12-26_Ti5-40xPh2/40x_DarkImage.tiff"

In [ ]:
hdf5_extractor.inter_set_flatfieldpaths()

##### Begin Extraction 

In [ ]:
hdf5_extractor.extract(dask_controller)

In [ ]:
dask_controller.shutdown()

## Kymographs

### Test Parameters



##### Initialize the interactive kymograph class

In [ ]:
interactive_kymograph = tr.kymograph_interactive(headpath)

In [ ]:
interactive_kymograph.import_hdf5_interactive()

##### Tune "trench-row" detection hyperparameters

In [ ]:
interactive_kymograph.preview_y_precentiles_interactive()

##### Tune "trench-row" cropping hyperparameters

In [ ]:
interactive_kymograph.preview_y_precentiles_consensus_interactive()

In [ ]:
interactive_kymograph.preview_y_crop_interactive()

##### Tune trench detection hyperparameters

In [ ]:
interactive_kymograph.preview_x_percentiles_interactive()

##### Tune trench cropping hyperparameters

In [ ]:
interactive_kymograph.preview_kymographs_interactive()

##### Export and save hyperparameters

In [ ]:
interactive_kymograph.process_results()

In [ ]:
interactive_kymograph.write_param_file()

### Generate Kymograph

##### Perform Kymograph Cropping

In [ ]:
kymoclust = tr.kymograph.kymograph_cluster(
    headpath=headpath, paramfile=True
)

##### Begin Kymograph Cropping 

In [ ]:
kymoclust.generate_kymographs(dask_controller)

In [ ]:
lane_overlay_handle = tr.lane_overlay(headpath,persist_data=True)
overlay = lane_overlay_handle.view_overlay(size=2500,vmin=0,vmax=4000)

In [ ]:
overlay

In [ ]:
lane_overlay_handle.save_rows()

##### Post-process Images

In [ ]:
kymoclust.post_process(dask_controller,trench_timepoints_per_file=10000)

##### Check kymograph statistics

In [ ]:
kymoclust.kymo_report()

#### Shutdown Dask

In [ ]:
dask_controller.shutdown()

## Fluorescence Segmentation (cells)

### Test Parameters

##### Initialize the interactive segmentation class

As a first step, initialize the `tr.fluo_segmentation_interactive` class that will be handling all steps of generating a segmentation. 

In [ ]:
interactive_segmentation = tr.fluo_segmentation_interactive(headpath)

##### Choose channel to segment on

In [ ]:
#maskpath is nucleoid only (for now)
interactive_segmentation.choose_seg_channel_inter()

#### Import data

In [ ]:
interactive_segmentation.import_array_inter()

##### Process data

In [ ]:
interactive_segmentation.plot_processed_inter()

#### Determine Cell Mask Envelope

In [ ]:
interactive_segmentation.plot_cell_mask_inter()

In [ ]:
interactive_segmentation.plot_eig_mask_inter()

In [ ]:
interactive_segmentation.plot_dist_mask_inter()

In [ ]:
interactive_segmentation.plot_marker_mask_inter()

In [ ]:
interactive_segmentation.process_results()

In [ ]:
interactive_segmentation.write_param_file()

### Generate Segmentation

#### Start Dask Workers

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="1:00:00",
    local=False,
    n_workers=500,
    n_workers_min=50,
    memory="1.5GB", # Use 1.5 GB next time
    working_directory=dask_wd,
)
dask_controller.startdask()

In [ ]:
dask_controller.displaydashboard()

In [ ]:
segment = tr.segment.fluo_segmentation_cluster(headpath, paramfile=True)

In [ ]:
segment.dask_segment(dask_controller,overwrite=False)

#### Stop Dask Workers

In [ ]:
dask_controller.shutdown()

## Lineage Tracing (cells)

### Test Parameters

In [ ]:
score_function = tr.tracking.scorefn(
    headpath,
    "fluorsegmentation",
    size_attr='axis_major_length',
    u_size=0.08,
    sig_size=0.03,
    u_pos=0.1,
    sig_pos=0.05,
    w_merge=0.,
)

In [ ]:
score_function.interactive_scorefn()

In [ ]:
Tracking_Solver = tr.tracking.tracking_solver(
    headpath, "fluorsegmentation", ScoreFn=score_function, edge_limit=2,
)
data, orientation = score_function.output.result

In [ ]:
Tracking_Solver.interactive_tracking(data, orientation)

In [ ]:
Tracking_Solver.save_params()

### Generate Lineage Traces

In [ ]:
# for general lineage trace

dask_controller = tr.trcluster.dask_controller(
    walltime="1:00:00",
    local=False,
    n_workers=200,
    n_workers_min=50,
    memory="4GB", # 2 GB with one channel
    working_directory=dask_wd,
)
dask_controller.startdask()

In [ ]:
dask_controller.daskclient

In [ ]:
Tracking_Solver = tr.tracking.tracking_solver(
    headpath, 
    "fluorsegmentation", 
    paramfile=True, 
    size_estimation=True,
    size_estimation_method='Perimeter/Area',
    props_list=['area', 'bbox', 'centroid'], 
    props_to_unpack={'bbox':('bbox_min_row', 'bbox_min_col', 'bbox_max_row', 'bbox_max_col'),
                     'centroid':('centroid_y', 'centroid_x')},
    pixel_scaling_factors={'area': 2,
                           'bbox_min_row' : 1, 'bbox_min_col' : 1,
                           'bbox_max_row' : 1, 'bbox_max_col' : 1, 
                           'centroid_y' : 1, 'centroid_x' : 1},
    intensity_props_list=['mean_intensity'] 
)

In [ ]:
Tracking_Solver.compute_all_lineages(dask_controller,entries_per_partition = 100000, overwrite=False)

#### Stop Dask Workers

In [ ]:
dask_controller.shutdown()

## Fluorescence Segmentation (nucleoids)

### Test Parameters

##### Initialize the interactive segmentation class

In [ ]:
interactive_segmentation = tr.fluo_segmentation_interactive(headpath)

##### Choose channel to segment on

In [ ]:
#maskpath is nucleoid only (for now)
#maskpath should be using the folder in which cell segments are contained
interactive_segmentation.choose_seg_channel_inter()

#### Import data

In [ ]:
interactive_segmentation.import_array_inter()

##### Process data

In [ ]:
interactive_segmentation.plot_processed_inter()

#### Determine Cell Mask Envelope

In [ ]:
interactive_segmentation.plot_cell_mask_inter()

In [ ]:
interactive_segmentation.plot_eig_mask_inter()

In [ ]:
interactive_segmentation.plot_dist_mask_inter()

In [ ]:
interactive_segmentation.plot_marker_mask_inter()

In [ ]:
interactive_segmentation.process_results()

In [ ]:
interactive_segmentation.write_param_file()

### Generate Segmentation

#### Start Dask Workers

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="1:00:00",
    local=False,
    n_workers=500,
    n_workers_min=50,
    memory="3GB", # Use 1.5 GB next time
    working_directory=dask_wd,
)
dask_controller.startdask()

In [ ]:
dask_controller.displaydashboard()

In [ ]:
segment = tr.segment.fluo_segmentation_cluster(headpath,
                                               segpath='nucleoidsegmentation',
                                               paramfile=True)

In [ ]:
segment.dask_segment(dask_controller,overwrite=True)

#### Stop Dask Workers

In [ ]:
dask_controller.shutdown()

## Regionprops (nucleoids)

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="1:00:00",
    local=False,
    n_workers=100,
    n_workers_min=100,
    memory="4GB", # Use 1.5 GB next time
    working_directory=dask_wd,
)
dask_controller.startdask()

In [ ]:
dask_controller.daskclient

In [ ]:
analyzer = tr.analysis.regionprops_extractor(headpath, 
                                             "nucleoidsegmentation", 
                                             intensity_channel_list=["mCherry"],
                                             props_list=['centroid', 'area', 'axis_major_length', 'axis_minor_length',
                                                         'feret_diameter_max', 'solidity', 'eccentricity', 'orientation'],
                                             intensity_props_list=['mean_intensity'],
                                             props_to_unpack={'centroid': ['centroid_y', 'centroid_x']},
                                             pixel_scaling_factors={'centroid_y': 1, 'centroid_x': 1, 'area': 2, 
                                                                    'axis_major_length': 1, 'axis_minor_length': 1, 
                                                                    'feret_diameter_max': 1}
)

In [ ]:
analyzer.analyze_all_files(dask_controller)

# Part 2: Barcodes

#### Specify Paths

In [ ]:
headpath = "/home/de64/scratch/de64/sync_folder/2023-05-06_lDE30_Run_1/Barcodes"
hdf5inputpath = "/home/de64/scratch/de64/sync_folder/2023-05-06_lDE30_Run_1/run/"

## Extract to hdf5 files

#### Start Dask Workers

In [ ]:
dask_controller = tr.trcluster.dask_controller(
    walltime="1:00:00",
    local=False,
    n_workers=200,
    n_workers_min=50,
    memory="2GB",
    working_directory=dask_wd
)
dask_controller.startdask()

In [ ]:
dask_controller.displaydashboard()

##### Perform Extraction

In [ ]:
hdf5_extractor = tr.marlin_extractor(hdf5inputpath, headpath, pixel_microns=0.105951961895293, metaparsestr='metadata_{timepoint:d}.hdf5')

##### Extraction Parameters

In [ ]:
hdf5_extractor.inter_set_params()

#### List of paths to flat fields
- /home/lag36/gutiloluis/FlatFields/2022-12-26_Ti5-40xPh2/40x_Cy5-Penta.tiff
- /home/lag36/gutiloluis/FlatFields/2022-12-26_Ti5-40xPh2/40x_Cy7-Penta.tiff
- /home/lag36/gutiloluis/FlatFields/2022-12-26_Ti5-40xPh2/40x_GFP-Penta.tiff
- /home/lag36/gutiloluis/FlatFields/2022-12-26_Ti5-40xPh2/40x_DarkImage.tiff

In [ ]:
hdf5_extractor.inter_set_flatfieldpaths()

##### Begin Extraction 

In [ ]:
hdf5_extractor.extract(dask_controller)

## Kymographs

### Test Parameters

##### Initialize the interactive kymograph class

In [ ]:
interactive_kymograph = tr.kymograph_interactive(headpath)

In [ ]:
interactive_kymograph.import_hdf5_interactive()

##### Tune "trench-row" detection hyperparameters

In [ ]:
interactive_kymograph.preview_y_precentiles_interactive()

In [ ]:
interactive_kymograph.preview_y_precentiles_consensus_interactive()

##### Tune "trench-row" cropping hyperparameters

In [ ]:
interactive_kymograph.preview_y_crop_interactive()

##### Tune trench detection hyperparameters

In [ ]:
interactive_kymograph.preview_x_percentiles_interactive()

##### Tune trench cropping hyperparameters

In [ ]:
interactive_kymograph.preview_kymographs_interactive()

##### Export and save hyperparameters

In [ ]:
interactive_kymograph.process_results()

In [ ]:
interactive_kymograph.write_param_file()

### Generate Kymograph

##### Perform Kymograph Cropping

In [ ]:
kymoclust = tr.kymograph.kymograph_cluster(
    headpath=headpath, paramfile=True
)

##### Begin Kymograph Cropping 

In [ ]:
kymoclust.generate_kymographs(dask_controller)

##### Post-process Images

In [ ]:
kymoclust.post_process(dask_controller,trench_timepoints_per_file=10000)

##### Check kymograph statistics

In [ ]:
kymoclust.kymo_report()

##### Shutdown Dask

In [ ]:
dask_controller.shutdown()

## FISH Analysis

#### Get Barcode Signal (Percentile Function)

In [ ]:
import numpy as np

tr.get_all_image_measurements(
    dask_controller,
    headpath,
    headpath + "/percentiles",
    ["GFP","Cy5","Cy7"],
    "98th Percentile",
    np.percentile,
    98)

#### Determine Barcodes

In [ ]:
fish_test = tr.fish_analysis(headpath,
                             "/home/lag36/scratch/lag36/2023-03-10_lDE26_Nanopore/lDE26_final_df.tsv",
                             remove_bit_list=[],
                             hamming_thr=2,\
                             channel_names=["GFP 98th Percentile","Cy5 98th Percentile","Cy7 98th Percentile"])

In [ ]:
fish_test.plot_signal_threshold_inter()

In [ ]:
fish_test.get_bit_thresholds()

In [ ]:
fish_test.plot_bit_threshold_inter()

In [ ]:
fish_test.export_bit_thresholds()

In [ ]:
fish_test.output_barcode_df(dask_controller,
                            '/home/de64/scratch/de64/sync_folder/2023-05-06_lDE30_Run_1/Barcodes/barcode_output_df.hdf5')

#### Shutdown Dask

In [ ]:
dask_controller.shutdown()